# Creating a new coupled system step by step

Connecting to the knowledge base

In [1]:
from coupled_modelling import KnowledgeBase

kb = KnowledgeBase('http://127.0.0.1:5000/api/v1.0/')

kb

Importing an existing coupled system from the JSON file

In [2]:
import os
import json

main_dir = os.path.dirname(os.path.abspath(''))

input_path = os.path.join(
    main_dir,
    'mixed_fid_models/',
    'ProjectParametersCoSimFSI.json')

with open(input_path) as f:
    data = json.load(f)

mixed_fid_models = kb.import_kratos('mixed_fid_models', data)

mixed_fid_models

Creating a new coupled system instance

In [3]:
Onera_FSI = kb.create_coupled('Onera_FSI')

Onera_FSI

We guess a new couple system is similar to *mixed_fid_models* and can reuse its properties

In [4]:
mixed_fid_models.get_properties()


{'label': 'mixed_fid_models',
 'problem_data': 'instance_79',
 'solver_settings': 'instance_80'}

Let's explore problem data of *mixed_fid_models*

In [5]:
mixed_fid_models.properties['problem_data']

'instance_79'

In [6]:
problem_data_old = kb.get_instance('instance_2')
problem_data_old.get_properties()

{'echo_level': 0,
 'end_time': 1.0,
 'parallel_type': 'OpenMP',
 'print_colors': True,
 'start_time': 0.0}

All values of the problem data from *mixed_fid_models* are similar to the new model, except of the *echo_level* value. We make a copy of the old problem data, set a new *echo_level* value and specify the new coupled system as a parent instance

In [7]:
problem_data_new = problem_data_old.make_copy(Onera_FSI, {'echo_level': 2})
problem_data_new.get_properties()

{'echo_level': 2,
 'end_time': 1.0,
 'parallel_type': 'OpenMP',
 'print_colors': True,
 'start_time': 0.0}

Let's explore *solver_settings* of the old model

In [8]:
mixed_fid_models.properties['solver_settings']

'instance_80'

In [9]:
solver_settings_old = kb.get_instance('instance_4')
solver_settings_old.get_properties()

{'convergence_accelerators': ['instance_10'],
 'convergence_criteria': ['instance_14'],
 'coupling_sequence': ['instance_16', 'instance_17'],
 'data_transfer_operators': 'instance_6',
 'echo_level': 3,
 'num_coupling_iterations': 20,
 'solvers': ['instance_23', 'instance_36'],
 'type': 'coupled_solvers.gauss_seidel_strong'}

By default, the copying method does not copy nested instances. Thus, we can copy first the solver settings from the existing coupled system with the old *num_coupling_iterations* and *type*, but with a new *echo_level*. Afterwards, we will create instances of convergence accelerators, convergence criteria, coupling sequence, data transfer operators and solvers, and add them to the mew solver settings

In [10]:
solver_settings_new = solver_settings_old.make_copy(Onera_FSI, {'echo_level': 4})
solver_settings_new.get_properties()

{'echo_level': 4,
 'num_coupling_iterations': 20,
 'type': 'coupled_solvers.gauss_seidel_strong'}

Checking old convergence accelerators

In [11]:
solver_settings_old.properties['convergence_accelerators']

['instance_10']

In [12]:
convergence_accelerators_old = kb.get_instance('instance_10')
convergence_accelerators_old.get_properties()

{'data_name': 'pitch_angle', 'solver': 'low_fid_fluid', 'type': 'aitken'}

Only *type* of the existing convergence accelerators can be reused, thus, we create a fresh new instance for new convergence accelerators and specify its class, parent instance and new property values

In [13]:
convergence_accelerators_new = kb.create_instance(
    'convergence_accelerators',
    solver_settings_new,
    {'data_name': 'displacements', 'solver': 'CFD', 'type': 'aitken'})

convergence_accelerators_new.get_properties()

{'data_name': 'displacements', 'solver': 'CFD', 'type': 'aitken'}

Copying convergence criteria, passing new *data_name*, *solver* and *type*

In [14]:
solver_settings_old.properties['convergence_criteria']

['instance_14']

In [15]:
convergence_criteria_old = kb.get_instance('instance_14')
convergence_criteria_new = convergence_criteria_old.make_copy(solver_settings_new, {
    'data_name': 'displacements',
    'solver': 'CFD',
    'type': 'relative_norm_initial_residual'})
convergence_criteria_new.get_properties()

{'abs_tolerance': 1e-07,
 'data_name': 'displacements',
 'rel_tolerance': 1e-05,
 'solver': 'CFD',
 'type': 'relative_norm_initial_residual'}

We create a new coupling sequence by creating its elements one by one

In [16]:
coupling_sequence_new_0 = kb.create_instance('coupling_sequence', solver_settings_new, {
    'name': 'CFD'})

coupling_sequence_new_0.get_properties()

{'name': 'CFD'}

Creating another coupling sequence

In [17]:
coupling_sequence_new_1 = kb.create_instance('coupling_sequence', solver_settings_new, {
    'name': 'SM'})

coupling_sequence_new_1.get_properties()

{'name': 'SM'}

Let's check properties of the second item of the old coupled sequence

In [18]:
solver_settings_old.properties['coupling_sequence'][1]

'instance_17'

In [19]:
coupling_sequence_old_1 = kb.get_instance('instance_17')
coupling_sequence_old_1.get_properties()

{'input_data_list': ['instance_19'],
 'name': 'spring_structure',
 'output_data_list': ['instance_22']}

We create new nested instances for the second item in the new coupling sequence by copying nested instances of *coupling_sequence_old_1*.

In [20]:
coupling_sequence_old_1.properties['input_data_list']

['instance_19']

In [21]:
input_data_list_old = kb.get_instance('instance_19')
input_data_list_new = input_data_list_old.make_copy(coupling_sequence_new_1, {
    'data_transfer_operator': 'mapping_operation',
    'from_solver': 'CFD',
    'data_transfer_operator_options': 'use_transpose'})
input_data_list_new.get_properties()

{'data': 'lift_force',
 'data_transfer_operator': 'mapping_operation',
 'data_transfer_operator_options': ['use_transpose'],
 'from_solver': 'CFD',
 'from_solver_data': 'lift_force'}

Creating a new output data list

In [22]:
coupling_sequence_old_1.properties['output_data_list']

['instance_22']

In [23]:
output_data_list_old = kb.get_instance('instance_22')
output_data_list_new = kb.create_instance('output_data_list', coupling_sequence_new_1, {
    'data': 'displacements',
    'data_transfer_operator': 'mapping_operation',
    'to_solver': 'CFD',
    'to_solver_data': 'displacements'})
output_data_list_new.get_properties()

{'data': 'displacements',
 'data_transfer_operator': 'mapping_operation',
 'to_solver': 'CFD',
 'to_solver_data': 'displacements'}

Creating a mapping operation instance as the data transfer operator

In [24]:
mapping_operation = kb.create_instance('data_transfer_operators', solver_settings_new, {
    'label': 'mapping_operation',
    'type': 'kratos_mapping'})
mapping_operation.get_properties()

{'label': 'mapping_operation', 'type': 'kratos_mapping'}

Creating new mapper settings

In [25]:
mapper_settings = kb.create_instance('mapper_settings', mapping_operation, {
    'mapper_type': 'nearest_neighbor',
    'use_initial_configuration': True})
mapper_settings.get_properties()

{'mapper_type': 'nearest_neighbor', 'use_initial_configuration': True}

Checking the first existing solver

In [26]:
solver_settings_old.properties['solvers'][0]

'instance_23'

In [27]:
solver_old_0 = kb.get_instance('instance_23')
solver_old_0.get_properties()

{'data': ['instance_31', 'instance_34'],
 'io_settings': 'instance_27',
 'label': 'low_fid_fluid',
 'solver_wrapper_settings': 'instance_25',
 'type': 'solver_wrappers.external.remote_controlled_solver_wrapper'}

In the new soupled system, we apply the CFD solver. Thus, we creat an instance of the CFD solver by copying the first solver

In [28]:
CFD = solver_old_0.make_copy(solver_settings_new, {
    'label': 'CFD'})
CFD.get_properties()

{'label': 'CFD',
 'type': 'solver_wrappers.external.remote_controlled_solver_wrapper'}

The old data doesn't fit to the new solver. Creating displacements data for the CFD solver instance

In [29]:
displacements_0 = kb.create_instance('data', CFD, {
    'label': 'displacements',
    'dimensions': 3,
    'model_part_name': 'WING',
    'variable_name': 'MESH_DISPLACEMENT'})
displacements_0.get_properties()

{'dimensions': 3,
 'label': 'displacements',
 'model_part_name': 'WING',
 'variable_name': 'MESH_DISPLACEMENT'}

Creating lift force data instance for the CFD solver

In [30]:
lift_force_0 = kb.create_instance('data', CFD, {
    'label': 'lift_force',
    'dimensions': 3,
    'model_part_name': 'WING',
    'variable_name': 'REACTION'})
lift_force_0.get_properties()

{'dimensions': 3,
 'label': 'lift_force',
 'model_part_name': 'WING',
 'variable_name': 'REACTION'}

Checking old *io_settings*

In [31]:
solver_old_0.properties['io_settings']

'instance_27'

In [32]:
io_settings_old = kb.get_instance('instance_27')
io_settings_old.get_properties()

{'communication_format': 'file',
 'connect_to': 'run_fluid',
 'echo_level': 4,
 'type': 'kratos_co_sim_io'}

Copying io settings with a new *connect_to* value

In [33]:
io_settings_new = io_settings_old.make_copy(CFD, {
    'connect_to': 'run_SU2'})
io_settings_new.get_properties()

{'communication_format': 'file',
 'connect_to': 'run_SU2',
 'echo_level': 4,
 'type': 'kratos_co_sim_io'}

Checking old solver wrapper settings

In [34]:
solver_old_0.properties['solver_wrapper_settings']

'instance_25'

In [35]:
solver_wrapper_settings_old_0 = kb.get_instance('instance_25')
solver_wrapper_settings_old_0.get_properties()

{'export_data': ['pitch_angle'],
 'import_data': ['lift_force'],
 'import_meshes': ['single_node_mesh_f']}

Copying solver wrapper settings

In [36]:
solver_wrapper_settings_new_0 = solver_wrapper_settings_old_0.make_copy(CFD, {
    'export_data': 'displacements',
    'import_meshes': 'WING'})
solver_wrapper_settings_new_0.get_properties()

{'export_data': ['displacements'],
 'import_data': ['lift_force'],
 'import_meshes': ['WING']}

Checking the second existing solver

In [37]:
solver_settings_old.properties['solvers'][1]

'instance_36'

In [38]:
solver_old_1 = kb.get_instance('instance_36')
solver_old_1.get_properties()

{'data': ['instance_41', 'instance_42'],
 'io_settings': 'instance_39',
 'label': 'spring_structure',
 'solver_wrapper_settings': 'instance_37',
 'type': 'solver_wrappers.external.remote_controlled_solver_wrapper'}

As the second solver for the new coupled system, we creat an instance of the SM solver

In [39]:
SM = kb.create_instance('solvers', solver_settings_new, {
    'label': 'SM',
    'type': 'solver_wrappers.kratos.structural_mechanics_wrapper'})
SM.get_properties()

{'label': 'SM', 'type': 'solver_wrappers.kratos.structural_mechanics_wrapper'}

Coping data for the second new solver from the first one

In [40]:
displacements_1 = displacements_0.make_copy(SM, {
    'model_part_name': 'Structure.interface',
    'variable_name': 'DISPLACEMENT'})
displacements_1.get_properties()

{'dimensions': 3,
 'label': 'displacements',
 'model_part_name': 'Structure.interface',
 'variable_name': 'DISPLACEMENT'}

Copying the second data item from the first solver

In [41]:
lift_force_1 = lift_force_0.make_copy(SM, {
    'model_part_name': 'Structure.interface',
    'variable_name': 'POINT_LOAD'})
lift_force_1.get_properties()

{'dimensions': 3,
 'label': 'lift_force',
 'model_part_name': 'Structure.interface',
 'variable_name': 'POINT_LOAD'}

Creating solver wrapper settings for the second new solver

In [42]:
solver_wrapper_settings_new_1 = kb.create_instance('solver_wrapper_settings', SM, {
    'input_file': 'ProjectParametersSM'})
solver_wrapper_settings_new_1.get_properties()

{'input_file': 'ProjectParametersSM'}

Exporting created coupled system in a Kratos-compatible JSON format

In [44]:
export = Onera_FSI.export_kratos()

export

{'problem_data': {'echo_level': 2,
  'end_time': 1.0,
  'parallel_type': 'OpenMP',
  'print_colors': True,
  'start_time': 0.0},
 'solver_settings': {'convergence_accelerators': [{'data_name': 'displacements',
    'solver': 'CFD',
    'type': 'aitken'}],
  'convergence_criteria': [{'abs_tolerance': 1e-07,
    'data_name': 'displacements',
    'rel_tolerance': 1e-05,
    'solver': 'CFD',
    'type': 'relative_norm_initial_residual'}],
  'coupling_sequence': [{'name': 'CFD'},
   {'input_data_list': [{'data': 'lift_force',
      'data_transfer_operator': 'mapping_operation',
      'data_transfer_operator_options': ['use_transpose'],
      'from_solver': 'CFD',
      'from_solver_data': 'lift_force'}],
    'name': 'SM',
    'output_data_list': [{'data': 'displacements',
      'data_transfer_operator': 'mapping_operation',
      'to_solver': 'CFD',
      'to_solver_data': 'displacements'}]}],
  'data_transfer_operators': {'mapping_operation': {'mapper_settings': {'mapper_type': 'nearest_nei

Saving exported data into a file

In [45]:
export_path = os.path.join(
    os.path.abspath(''),
    'export_onera_fsi.json')

with open(export_path, 'w') as file:
    json.dump(export, file, indent=2)

Once the coupled system is resolved, we can specify a link to the result

In [43]:
Onera_FSI.add_result('https://github.com/ldrbmrtv/coupled_modelling/tree/main/examples/Onera_FSI/results')

''

Inferring new classes from the created coupled system

In [46]:
Onera_FSI.infer_classes()

''

To check the infered classes, we can retrieve the class hierarchy of the knowledge base

In [47]:
kb.get_class_hierarchy()

{'communication_format': [],
 'connect_to': [],
 'convergence_accelerators': ['convergence_accelerators_1',
  'convergence_accelerators_2'],
 'convergence_criteria': ['convergence_criteria_1'],
 'coupled_system': ['coupled_system_1', 'coupled_system_2'],
 'coupling_sequence': ['coupling_sequence_1',
  'coupling_sequence_2',
  'coupling_sequence_3'],
 'data': ['data_1', 'data_2', 'data_3'],
 'data_name': [],
 'data_transfer_operator': [],
 'data_transfer_operator_options': [],
 'data_transfer_operators': ['data_transfer_operators_1',
  'data_transfer_operators_2'],
 'export_data': [],
 'from_solver': [],
 'from_solver_data': [],
 'import_data': [],
 'import_meshes': [],
 'input_data_list': ['input_data_list_1'],
 'input_file': [],
 'io_settings': ['io_settings_1'],
 'location': [],
 'mapper_settings': ['mapper_settings_1', 'mapper_settings_2'],
 'mapper_type': [],
 'model_part_name': [],
 'name': [],
 'output_data_list': ['output_data_list_1', 'output_data_list_2'],
 'parallel_type': []